<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60">

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'>


Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego 
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej".   
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

# Statistical machine learning - Notebook 10, version for students 
**Author: Michał Ciach**  
**Date: 10.12.2021**

## Description
 

In this class, we will learn the basics of two popular classification algorithms: the Logistic Regression and the Linear Discriminant Analysis.

In [1]:
!pip install gdown
!gdown https://drive.google.com/uc?id=1GW1pjKOCoKOlC4Jqbqql_ghYD_n0iC6O
!gdown https://drive.google.com/uc?id=1FInZ2jrlZGNColU4sHF9JKGHP39fTVut
!gdown https://drive.google.com/uc?id=1n1qS6dcVVKcVJOuUIIm0VTz6cSyrtzDH

Downloading...
From: https://drive.google.com/uc?id=1GW1pjKOCoKOlC4Jqbqql_ghYD_n0iC6O
To: /content/BDL municipality incomes 2015-2020.csv
100% 228k/228k [00:00<00:00, 82.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1FInZ2jrlZGNColU4sHF9JKGHP39fTVut
To: /content/BDL municipality area km2 2015-2020.csv
100% 180k/180k [00:00<00:00, 51.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1n1qS6dcVVKcVJOuUIIm0VTz6cSyrtzDH
To: /content/BDL municipality population 2015-2020.csv
100% 222k/222k [00:00<00:00, 76.8MB/s]


## Data & library imports

In [2]:
import pandas as pd
import plotly.express as px
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.decomposition import PCA
from sklearn import datasets

In [3]:
income = pd.read_csv('BDL municipality incomes 2015-2020.csv', sep=';', dtype={'Code': 'str'})
population = pd.read_csv('BDL municipality population 2015-2020.csv', sep='\t', dtype={'Code': 'str'})
area = pd.read_csv('BDL municipality area km2 2015-2020.csv', sep='\t', dtype={'Code': 'str'})

In [4]:
voivodeship_names = {
    '02': 'Dolnośląskie',
    '04': 'Kujawsko-pomorskie',
    '06': 'Lubelskie',
    '08': 'Lubuskie',
    '10': 'Łódzkie',
    '12': 'Małopolskie',
    '14': 'Mazowieckie',
    '16': 'Opolskie',
    '18': 'Podkarpackie',
    '20': 'Podlaskie',
    '22': 'Pomorskie',
    '24': 'Śląskie',
    '26': 'Świętokrzyskie',
    '28': 'Warmińsko-mazurskie',
    '30': 'Wielkopolskie',
    '32': 'Zachodniopomorskie'
}

In [5]:
code_list = [s[:2] for s in income["Code"]]
name_list = [voivodeship_names[code] for code in code_list]
income['Voivodeship'] = name_list

## Part 1

**Exercise 1.** In this exercise, we will use logistic regression to classify iris flowers into species based on their measurements. We will also analyze the sensitivity of this model to errors in the training data set. 

The `iris` data set is loaded in the first cell below. The vector `dataset` assigns each observation either to the test or the training data set. Additionally, the vector `y_ptb` has 20% of species assigned incorrectly to stimulate errors in the training data set. 

In the next cell, three artificial observations are created. They are located near the boundaries of different species. 

First, use the principal component analysis to show the distribution of flower measurements on a scatter plot. Color the points according to their species. Next, color the points according to the labeling with errors to see how it changes the structure of the data set. Add points from the `X_new` array to the plots to see where they're located. 

Split the `X` and `y` arrays into test and training data sets. Try to do this using the `pd.groupby()` and the `pd.get_group()` functions. 

Fit a logistic regression model to the training data set. Check the training and the test accuracy of the fitted model. Check the coefficients of the model. What measurements increase the probability that a flower is classified as *Iris setosa*, and which do decrease it? Are all the variables used to classify the flowers? 

Predict the classes for the data points in `X_new`. Calculate the probabilities of assignment to different classes. Are they similar to the weights used to create them? Is the model confident about the classification of those points? 

Fit the model to the training data set with errors. Calculate the training and the test accuracy (use the test data set without errors). Did the errors influence the quality of fit? Is the model fitted to low-quality data equally able to correctly classify new observations in this case? How did the errors influence the model's confidence of classification of the artificial observations from `X_new`?

In [6]:
iris_raw = datasets.load_iris()
iris = pd.DataFrame(iris_raw.data, columns=iris_raw.feature_names)
iris['Species'] = [iris_raw.target_names[x] for x in iris_raw.target]

X = iris.iloc[:, :4]
y = iris['Species']
dataset = ["Test" if not i % 10 else "Train" for i in range(X.shape[0])]
y_ptb = [y.iloc[(i+50)%150 ] if not (i-1)%10 else y.iloc[(i+100)%150] if not (i-2)%10 else y.iloc[i] for i in range(len(y))]

In [7]:
specie_centers = iris.groupby("Species").mean()
nwpt_prop = np.array([
            [0.00, 0.50, 0.50],
            [0.60, 0.40, 0.00],
            [0.33, 0.33, 0.33]
             ]) 
X_new = nwpt_prop.dot(specie_centers)

In [107]:
X_new

array([[6.262  , 2.872  , 4.906  , 1.676  ],
       [5.378  , 3.1648 , 2.5812 , 0.678  ],
       [5.7849 , 3.02676, 3.72042, 1.18734]])

In [117]:
pca = PCA()
components = pca.fit_transform(X)

In [27]:
import plotly.graph_objects as go

In [118]:
labels = {
    str(i): f"PC {i+1} ({var:.1f}%)"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
}


fig = px.scatter_matrix(
    components,
    labels=labels,
    dimensions=range(4),
    color=y
)

fig.update_traces(diagonal_visible=False)
fig.show()

In [119]:
# fig = px.scatter(components, x=0, y=1, color=y)
# fig.add_trace(go.Scatter(x=X_new[:, 0], y=X_new[:, 1], mode='markers', marker=go.scatter.Marker(symbol='x',size=12, color = 'black'), showlegend=False))
# fig.show()

In [26]:
labels = {
    str(i): f"PC {i+1} ({var:.1f}%)"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
}

fig = px.scatter_matrix(
    components,
    labels=labels,
    dimensions=range(4),
    color=y_ptb
)
fig.update_traces(diagonal_visible=False)
fig.show()

In [29]:
df = {'y': y, 'set': dataset, 'y_ptb': y_ptb}
df = pd.DataFrame(df)
data = pd.concat([X, df], axis=1)

In [31]:
grouped = data.groupby(['set'])
train = grouped.get_group('Train')
test = grouped.get_group('Test')

In [32]:
train

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),y,set,y_ptb
1,4.9,3.0,1.4,0.2,setosa,Train,versicolor
2,4.7,3.2,1.3,0.2,setosa,Train,virginica
3,4.6,3.1,1.5,0.2,setosa,Train,setosa
4,5.0,3.6,1.4,0.2,setosa,Train,setosa
5,5.4,3.9,1.7,0.4,setosa,Train,setosa
...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica,Train,virginica
146,6.3,2.5,5.0,1.9,virginica,Train,virginica
147,6.5,3.0,5.2,2.0,virginica,Train,virginica
148,6.2,3.4,5.4,2.3,virginica,Train,virginica


In [35]:
features = ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']

In [39]:
clf = LogisticRegression(max_iter=200).fit(train[features], train['y'])

In [45]:
acc_train = clf.score(train[features], train['y'])
acc_train

0.9851851851851852

In [44]:
acc_test = clf.score(test[features], test['y'])
acc_test

0.9333333333333333

In [46]:
clf.coef_

array([[-0.43765907,  0.91090199, -2.44914142, -1.04965943],
       [ 0.59805841, -0.35844208, -0.22048899, -0.92943996],
       [-0.16039934, -0.55245991,  2.66963041,  1.97909939]])

In [48]:
clf.predict(X_new)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning:

X does not have valid feature names, but LogisticRegression was fitted with feature names



array(['virginica', 'setosa', 'versicolor'], dtype=object)

In [49]:
clf.predict_proba(X_new)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning:

X does not have valid feature names, but LogisticRegression was fitted with feature names



array([[1.02157199e-03, 4.73556971e-01, 5.25421457e-01],
       [6.10478252e-01, 3.89468845e-01, 5.29032110e-05],
       [5.95573522e-02, 9.29170887e-01, 1.12717609e-02]])

In [51]:
clf = LogisticRegression(max_iter=200).fit(train[features], train['y_ptb'])

In [52]:
acc_train = clf.score(train[features], train['y_ptb'])
acc_train

0.7185185185185186

In [55]:
acc_train = clf.score(train[features], train['y'])
acc_train

0.8740740740740741

In [57]:
acc_test = clf.score(test[features], test['y'])
acc_test

0.8666666666666667

In [56]:
acc_test = clf.score(train[features], train['y_ptb'])
acc_test

0.7185185185185186

In [58]:
clf.predict(X_new)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning:

X does not have valid feature names, but LogisticRegression was fitted with feature names



array(['virginica', 'setosa', 'versicolor'], dtype=object)

In [59]:
clf.predict_proba(X_new)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning:

X does not have valid feature names, but LogisticRegression was fitted with feature names



array([[0.11883888, 0.4091479 , 0.47201322],
       [0.52536121, 0.32053184, 0.15410695],
       [0.2833375 , 0.40549335, 0.31116915]])

**Exercise 2.** Use the Linear Discriminant Analysis to classify the iris flowers and compare it to the results from Exercise 1. Answer the following questions:

- Which classifier fits better to the training dataset? Which generalizes better to the test data set? 
- Which classifier shows a higher certainty (reflected by the class probabilities) when classifying new points? Is this a good or a bad feature?
- Which classifier is more robust to errors in the training data set?

In [60]:
clf = LinearDiscriminantAnalysis()
clf.fit(train[features], train['y'])

LinearDiscriminantAnalysis()

In [61]:
clf.score(train[features], train['y'])

0.9851851851851852

In [62]:
clf.score(test[features], test['y'])

0.9333333333333333

In [63]:
clf.predict_proba(test[features])

array([[1.00000000e+00, 7.94325302e-22, 5.18975432e-42],
       [1.00000000e+00, 2.51307253e-23, 6.23655082e-44],
       [1.00000000e+00, 1.53060601e-19, 4.09821292e-39],
       [1.00000000e+00, 3.18785546e-16, 1.02466197e-34],
       [1.00000000e+00, 2.55080105e-21, 4.93567237e-41],
       [5.59067780e-18, 9.99923425e-01, 7.65752261e-05],
       [7.58574326e-18, 9.99998685e-01, 1.31468535e-06],
       [2.19886074e-28, 1.37652935e-01, 8.62347065e-01],
       [2.99353635e-17, 9.99997188e-01, 2.81164714e-06],
       [9.67648352e-23, 9.99081339e-01, 9.18660691e-04],
       [1.01430308e-52, 2.38471544e-09, 9.99999998e-01],
       [3.87642626e-32, 9.29181713e-03, 9.90708183e-01],
       [1.06866935e-42, 4.87781044e-06, 9.99995122e-01],
       [3.61959649e-41, 2.05148471e-04, 9.99794852e-01],
       [5.38335109e-45, 8.37403169e-07, 9.99999163e-01]])

In [64]:
clf.fit(train[features], train['y_ptb'])

LinearDiscriminantAnalysis()

In [65]:
clf.score(train[features], train['y'])

0.8666666666666667

In [66]:
clf.score(test[features], test['y'])

0.8666666666666667

In [67]:
clf.predict_proba(test[features])

array([[0.90461395, 0.07182323, 0.02356283],
       [0.93583598, 0.04578602, 0.01837799],
       [0.87904318, 0.09849871, 0.02245811],
       [0.74447451, 0.21420997, 0.04131552],
       [0.90881735, 0.06467831, 0.02650434],
       [0.25092862, 0.42729685, 0.32177454],
       [0.04557093, 0.85794517, 0.0964839 ],
       [0.04480784, 0.20243828, 0.75275388],
       [0.08629768, 0.74732303, 0.16637929],
       [0.04540644, 0.64740889, 0.30718467],
       [0.00556613, 0.06071086, 0.93372301],
       [0.05766921, 0.20892832, 0.73340247],
       [0.03075567, 0.16074613, 0.80849821],
       [0.03539175, 0.52683467, 0.43777358],
       [0.02432777, 0.14932804, 0.82634419]])

**Exercise 3 (homework).** Use the available data and at least two statistical techniques (including at least one classification algorithm) to analyze what kinds of municipalities had a lower income in 2020 compared to 2019. 

In the cell below you have some initial preprocessing of the data that you may use for this exercise.

In [68]:
data = pd.merge(income, population.drop(columns='Region'), on='Code', suffixes=['income', 'population'])
data.dropna(inplace=True)
## Remove rows with 0 values (Boguszów-Gorce):
data = data[(data!=0).all(axis=1)] 
## Encode voivodeships with dummy variables:
data = pd.get_dummies(data, columns=['Voivodeship'])
## Create the response variable:
data['2020increase'] = data['2020income'] > data['2019income']
## Store the data about regions of rows in a separate table:
regions = data[['Code', 'Region']]
## Remove the regions and codes:
data = data.drop(columns=['2020income', '2020population', 'Code', 'Region'])
## Transform income and population columns:
columns_to_transform = data.filter(regex='income|population').columns
data[columns_to_transform] = np.log10(data[columns_to_transform])

In [72]:
cols = [x for x in data.columns if x != '2020increase']

In [74]:
from sklearn.model_selection import train_test_split

In [75]:
x_train, x_test, y_train, y_test = train_test_split(data[cols], data['2020increase'], test_size=0.33)

In [76]:
clf = LinearDiscriminantAnalysis()
clf.fit(x_train, y_train)

LinearDiscriminantAnalysis()

In [77]:
clf.score(x_test, y_test)

0.872366790582404

In [79]:
clf.coef_

array([[ 2.09949474e+00,  6.69537421e+00,  5.21181356e+00,
         5.05332992e+00, -2.51286398e+01, -4.92735837e+01,
         3.58332078e+01,  2.42280797e+01, -1.26209224e+02,
         1.21198660e+02,  2.87099773e-01, -2.85977492e-01,
        -2.61184335e-01, -6.32038149e-01,  3.53152177e-01,
         1.41063068e-01, -6.71025023e-01,  1.49679565e-01,
         4.61689979e-02,  9.41140859e-02, -3.41254641e-01,
         3.80417724e-01, -7.62012429e-02, -8.33957193e-01,
         6.08203906e-01, -1.57005225e-01]])

In [80]:
clf = LogisticRegression(max_iter=200).fit(x_train, y_train)

In [81]:
clf.score(x_test, y_test)

0.8674101610904585

In [82]:
clf.coef_

array([[ 0.64256484,  0.92210199,  0.95574606,  0.17202676, -5.41276174,
         0.15353603,  0.32277544,  0.48107382,  0.6384895 ,  0.88422445,
         0.32234179, -0.10358152, -0.19316061, -0.54447042,  0.51288094,
         0.57632955, -0.43564386,  0.22104473, -0.57028818,  0.26195447,
        -0.30329952,  0.45449397,  0.08891051, -0.64971673,  0.66561107,
        -0.26076586]])

<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60">

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'>


Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego 
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej".   
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>